# Vertica `TIMESERIES` 
> "Provides gap-filling and interpolation (GFI) computation, an important component of time series analytics computation."

## Syntax: 
```
TIMESERIES slice_time AS 'length_and_time_unit_expression' OVER ( 
... [ window_partition_clause  [ , ... ] ]  
... ORDER BY time_expression ) 
... [ ORDER BY table_column [ , ... ] ]
```

## 1. Connect to the Database
(be sure you have a running Vertica database at `localhost:5433`)

In [17]:
import vertica_python
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

connection_params = {
    'host' : 'localhost',
    'port' : 5433,
    'database' : 'docker',
    'user' : 'dbadmin',
    'password' : ""
}

connection = vertica_python.connect(**connection_params)
cur = connection.cursor()


## 2. Load the Data into Vertica
We're loading a dataset of mallard duck GPS location tracks from a several-month period. The data is accessible from [this link](https://s3-us-west-2.amazonaws.com/cse599c-sp17/mallard.csv). We'll download it here: 

In [18]:
%%bash 
curl -O https://s3-us-west-2.amazonaws.com/cse599c-sp17/mallard.csv
ls mallard.csv

mallard.csv


  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 8479k  100 8479k    0     0   9.9M      0 --:--:-- --:--:-- --:--:-- 10.0M


I've already written a schema for this file, let's look at it:


In [19]:
%%bash 
more createMallardsTable.sql

create table if not exists mallards (
	eventId varchar(20) PRIMARY KEY,
	ts timestamp,
	locationLong float,
	locationLat float);



We'll run the schema and load the data from the downloaded file.

In [20]:
tablefile = "createMallardsTable.sql"
with open(tablefile,'r') as createTable:
    cur.execute(createTable.read())
    
mallardfile = "mallard.csv"

cur = connection.cursor()
with open(mallardfile, "rb") as f:
    cur.copy("COPY mallards from stdin PARSER fcsvparser()", f)

And test it out: 

In [24]:
cur.execute("select * from mallards limit 10")
print(cur.fetchall())

[['1122500109', datetime.datetime(2012, 8, 24, 10, 16, 18), 4.723377, 51.876064], ['1122500110', datetime.datetime(2012, 8, 24, 10, 31, 41), 4.723216, 51.876404], ['1122500111', datetime.datetime(2012, 8, 24, 11, 6), 4.722925, 51.877159], ['1122500112', datetime.datetime(2012, 8, 24, 11, 45, 53), 4.72329, 51.876339], ['1122500113', datetime.datetime(2012, 8, 24, 12, 21, 22), 4.72336, 51.876328], ['1122500114', datetime.datetime(2012, 8, 24, 12, 59, 54), 4.723376, 51.876324], ['1122500115', datetime.datetime(2012, 8, 24, 13, 16, 8), 4.723266, 51.876259], ['1122500116', datetime.datetime(2012, 8, 24, 13, 50, 20), 4.723662, 51.876328], ['1122500117', datetime.datetime(2012, 8, 24, 14, 13, 26), 4.72322, 51.876625], ['1122500118', datetime.datetime(2012, 8, 24, 14, 51, 7), 4.723375, 51.876404]]


## 2. Develop some `TIMESERIES` Queries

### Primitives:
* `ts_first_value(<column>, '{'const', 'linear'}')` : provides the first value in a timeslice for a given column using the given interpolation strategy.
* `ts_last_value(<column>, '{'const', 'linear}')`: same as above but with last value. 

In [27]:
cur.execute(
    "select slice_time, ts_first_value(locationLong, 'const'), ts_first_value(locationLat, 'const')\
    from mallards\
    where ts between timestamp '2012-08-24 10:00' and timestamp '2012-08-24 17:00'\
    TIMESERIES slice_time AS '30 seconds' OVER (ORDER BY ts);"
)
df_const = pd.DataFrame(cur.fetchall())
df_const.head(3)


,0,1,2
0,2012-08-24 10:05:00,NaN,NaN
1,2012-08-24 10:05:30,4.724853,51.878796
2,2012-08-24 10:06:00,4.724853,51.878796


It works!

In [28]:
cur.execute(
    "select slice_time, ts_first_value(locationLong, 'linear'), ts_first_value(locationLat, 'linear')\
    from mallards\
    where ts between timestamp '2012-08-24 10:00' and timestamp '2012-08-24 17:00'\
    TIMESERIES slice_time AS '30 seconds' OVER (ORDER BY ts);"
)
df_linear = pd.DataFrame(cur.fetchall())
df_linear.head(3)

,0,1,2
0,2012-08-24 10:05:00,NaN,NaN
1,2012-08-24 10:05:30,4.723781,51.878684
2,2012-08-24 10:06:00,4.719763,51.878265


In [ ]:
cur.execute(
    "select ts, locationLong, locationLat\
    from mallards\
    where ts between timestamp '2012-08-24 10:00' and timestamp '2012-08-24 17:00'\
)
df_actual = pd.DataFrame(cur.fetchall())


In [ ]:
df_actual

In [ ]:
df_const


In [ ]:
df_linear

In [ ]:
df_linear[1].plot(marker=".")


In [ ]:
df_const[1].plot(marker=".")

In [ ]:
df_const[1].plot

In [ ]:
plt.plot(df_const[2], df_const[1])

In [ ]:
plt.plot(df_linear[2], df_linear[1])